In [1]:
%load_ext autoreload
%autoreload 2  

In [2]:
import matplotlib.pyplot as plt 

In [3]:
from torch.utils.tensorboard import SummaryWriter

In [4]:
import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from quickannotator.dl.inference import run_inference, getPendingInferenceTiles
from quickannotator.dl.dataset import TileDataset
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from safetensors.torch import save_file

def get_transforms(tile_size): #probably goes...elsewhere
    transforms = A.Compose([
    A.RandomScale(scale_limit=0.1, p=.9),
    A.PadIfNeeded(min_height=tile_size, min_width=tile_size),
    A.VerticalFlip(p=.5),
    A.HorizontalFlip(p=.5),
    A.Blur(p=.5),
    # Downscale(p=.25, scale_min=0.64, scale_max=0.99),
    A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),
    A.GridDistortion(p=.5, num_steps=5, distort_limit=(-0.3, 0.3),
                    border_mode=cv2.BORDER_REFLECT),
    A.ISONoise(p=.5, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
    A.RandomBrightnessContrast(p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
    A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),
    A.MultiplicativeNoise(p=.5, multiplier=(0.9, 1.1), per_channel=True, elementwise=True),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=10, val_shift_limit=10, p=.9),
    A.Rotate(p=1, border_mode=cv2.BORDER_REFLECT),
    A.RandomCrop(tile_size, tile_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalization
    ToTensorV2()])
    return transforms



/opt/uv_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
classid = 2 
tile_size = 2_048 

boost_count = 5
batch_size_train=1
batch_size_infer=1
edge_weight=1_000
num_workers=0 

num_iters=50

dataset=TileDataset(classid, tile_size=tile_size, 
                    edge_weight=edge_weight, transforms=get_transforms(tile_size), 
                    boost_count=boost_count)

dataloader = DataLoader(dataset, batch_size=batch_size_train, shuffle=False, num_workers=num_workers) #NOTE: for dataset of type iter - shuffle must == False

model = smp.Unet(encoder_name="tu-convnext_tiny", encoder_weights="imagenet", in_channels=3, classes=1 )
criterion = nn.BCEWithLogitsLoss(reduction='none', )
optimizer = optim.NAdam(model.parameters(), lr=0.001, weight_decay=1e-2) #TODO: this should be a setting

device = 'cuda' if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()



/tmp/ipykernel_23565/2896273232.py:27: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),
/tmp/ipykernel_23565/2896273232.py:28: UserWarning: Argument(s) 'border_mode' are not valid for transform GridDistortion
  A.GridDistortion(p=.5, num_steps=5, distort_limit=(-0.3, 0.3),
/tmp/ipykernel_23565/2896273232.py:32: UserWarning: Argument(s) 'eps' are not valid for transform RandomGamma
  A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),


Unet(
  (encoder): TimmUniversalEncoder(
    (model): FeatureListNet(
      (stem_0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (stem_1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      (stages_0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (norm): Identity()
              (fc2): Linear(in_features=384, out_features=96, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv2d(9

In [ ]:
import datetime
running_loss = []
writer = SummaryWriter(log_dir=f"/tmp/{classid}/{datetime.datetime.now().strftime('%b%d_%H-%M-%S')}")

for niter in tqdm(range(num_iters*5)): #TODO: this should be a setting
    images, masks, weights = next(iter(dataloader))
    #print ("post next iter")
    images = images.to(device) #TODO: test with .half()
    masks = masks.to(device)
    weights = weights.to(device)
    #print ("post copy ")
    for _ in tqdm(range(1)):
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, masks.float())
        loss = (loss * (edge_weight ** weights).type_as(loss)).mean()

        positive_mask = (masks == 1).float()
        unlabeled_mask = (masks == 0).float()

        positive_loss = 1.0 * (loss * positive_mask).mean()
        unlabeled_loss = .1 * (loss * unlabeled_mask).mean()

        loss_total = positive_loss + unlabeled_loss
        loss_total.backward()

        optimizer.step()

    running_loss.append(loss_total.item())

    writer.add_scalar(f'loss/loss', loss, niter)
    writer.add_scalar(f'loss/positive_loss', positive_loss, niter)
    writer.add_scalar(f'loss/unlabeled_loss', unlabeled_loss, niter)
    writer.add_scalar(f'loss/loss_total', loss_total, niter)
    
    print ("losses:\t",loss_total,positive_mask.sum(),positive_loss,unlabeled_loss)

    
    if niter % 50==0:
        print (f"niter [{niter}], Loss: {sum(running_loss)/len(running_loss)}")
        running_loss=[]

        print ("saving!") #TODO: do we want to *always* override the last saved model , or do we want to instead only save if some type of loss threshold is met?
                            #another potentially more interesting option is to do both, save on a regular basis (since if we things crash we can revert back othe nearest checkpoint\
                            #but as well give the user in the front end a dropdown which enables them to select which model checkpoint they want to use? we had somethng similar in QAv1
                            #that said, this is likely a more advanced features and not very "apple like" since it would require explaining to the user when/why/how they should use the different models
                            #maybe suggest avoiduing for now --- lets just save the last one
        save_file(model.state_dict(), f"/tmp/model_{classid}.safetensors") #TODO: needs to go somewhere reasonable maybe /projid/models/classid/ ? or something
        last_save = 0



  0%|          | 0/250 [00:00<?, ?it/s]InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


tile retval <quickannotator.db.models.Tile object at 0x7fa160114c70>
tile retval 2 <quickannotator.db.models.Tile object at 0x7fa160114c70>


100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


losses:	 tensor(0.8827, device='cuda:0', grad_fn=<AddBackward0>) tensor(871649., device='cuda:0') tensor(0.6391, device='cuda:0', grad_fn=<MulBackward0>) tensor(0.2436, device='cuda:0', grad_fn=<MulBackward0>)
niter [0], Loss: 0.882665753364563
saving!


  0%|          | 1/250 [00:04<18:11,  4.38s/it]/opt/QuickAnnotator/quickannotator/db/utils.py:6: SAWarning: This declarative base already contains a class with the same class name and module name as quickannotator.db.utils.DynamicAnnotation, and will be replaced in the string-lookup table.
  class DynamicAnnotation(base):


...

losses:	 tensor(0.0485, device='cuda:0', grad_fn=<AddBackward0>) tensor(496151., device='cuda:0') tensor(0.0278, device='cuda:0', grad_fn=<MulBackward0>) tensor(0.0207, device='cuda:0', grad_fn=<MulBackward0>)
tile retval <quickannotator.db.models.Tile object at 0x7fa1310ec130>
tile retval 2 <quickannotator.db.models.Tile object at 0x7fa1310ec130>


100%|█████████▉| 249/250 [13:14<00:03,  3.28s/it]

losses:	 tensor(0.1365, device='cuda:0', grad_fn=<AddBackward0>) tensor(1143458., device='cuda:0') tensor(0.1078, device='cuda:0', grad_fn=<MulBackward0>) tensor(0.0286, device='cuda:0', grad_fn=<MulBackward0>)
tile retval <quickannotator.db.models.Tile object at 0x7fa13109bf10>
tile retval 2 <quickannotator.db.models.Tile object at 0x7fa13109bf10>


100%|██████████| 250/250 [13:17<00:00,  3.19s/it]

losses:	 tensor(0.0203, device='cuda:0', grad_fn=<AddBackward0>) tensor(0., device='cuda:0') tensor(0., device='cuda:0', grad_fn=<MulBackward0>) tensor(0.0203, device='cuda:0', grad_fn=<MulBackward0>)


In [ ]:
images.shape

In [ ]:
tds=TileDataset(classid, tile_size=tile_size, magnification=magnification,
                    edge_weight=edge_weight, transforms=None, 
                    boost_count=boost_count)


In [ ]:
images, masks, weights= next(iter(tds))
plt.imshow(images)

In [ ]:
images, masks, weights = next(iter(dataloader))
i=images.cpu().detach()
plt.imshow(i.squeeze().permute(1,2,0))


In [ ]:
plt.imshow(masks.cpu().detach().squeeze())  

In [ ]:
outputs = model(images.to(device))

In [ ]:
o=outputs.cpu().detach()

In [ ]:
plt.imshow(o.squeeze()>-3)

In [ ]:
plt.hist(o.squeeze().numpy().flatten())

In [ ]:
masks.shape